In [23]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt
import os

from sklearn.model_selection import train_test_split
import sklearn

Inserting all features from test into one dataframe

In [24]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)
        
        

       average_income_per_grunnkrets  C_1.1.1.0  C_1.1.2.0  C_1.1.3.0  \
0                           561700.0          1          0          0   
1                           555720.0          1          0          0   
2                           495900.0          1          0          0   
3                           521240.0          1          0          0   
4                           585360.0          1          0          0   
...                              ...        ...        ...        ...   
12854                       505640.0          0          0          0   
12855                       396640.0          0          0          0   
12856                       549120.0          0          0          0   
12857                       498660.0          0          0          0   
12858                       579780.0          0          0          0   

       C_1.1.4.0  C_1.1.5.0  C_1.1.6.1  C_1.1.6.2  C_1.1.6.3  C_1.1.6.4  ...  \
0              0          0          0     

Repeat for test features

In [25]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)
    

      average_income_per_grunnkrets  C_1.1.1.0  C_1.1.2.0  C_1.1.3.0  \
0                          551600.0          1          0          0   
1                          531240.0          1          0          0   
2                          523300.0          1          0          0   
3                          591380.0          1          0          0   
4                          594020.0          1          0          0   
...                             ...        ...        ...        ...   
8572                       560700.0          0          0          0   
8573                       593340.0          0          0          0   
8574                       629100.0          0          0          0   
8575                       590100.0          0          0          0   
8576                       529980.0          0          0          0   

      C_1.1.4.0  C_1.1.5.0  C_1.1.6.1  C_1.1.6.2  C_1.1.6.3  C_1.1.6.4  ...  \
0             0          0          0          0        

Target train data:

In [26]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

Light GBM model:

In [27]:
lgbm_model = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [28]:
#print(features_train)
#print(type(features_train))
print(len(features_train))
print(target_train)
print(type(target_train))
target_train = target_train
print(len(target_train))

12859
[2.94433371 3.21197205 2.83901998 ... 3.6693143  1.53514531 1.20237152]
<class 'numpy.ndarray'>
12859


In [29]:
lgbm_model.fit(features_train, target_train)
lgbm_predictions = lgbm_model.predict(features_test)

#Undo log transform
lgbm_predictions = np.expm1(lgbm_predictions)

c:\Users\47457\AppData\Local\Programs\Python\Python38\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


In [30]:
print(lgbm_predictions)

[5.20732714 5.73043364 5.56064084 ... 3.35088065 3.32725379 1.87106726]


In [31]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [32]:
predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = lgbm_predictions
predictions.to_csv("../predictions/lightGBM19.csv", index=False)
print(predictions)


                              id  predicted
0     914206820-914239427-717245   5.207327
1     916789157-916823770-824309   5.730434
2       913341082-977479363-2948   5.560641
3      889682582-889697172-28720   4.105838
4     997991699-998006945-417222   4.233617
...                          ...        ...
8572  917323003-917383529-844309   2.018843
8573  917353379-917411824-845904   3.161578
8574  917072302-917089248-833647   3.350881
8575  916960557-916993161-829908   3.327254
8576   987280891-972040746-45320   1.871067

[8577 rows x 2 columns]


In [33]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))


X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

lgbm_model2 = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

lgbm_model2.fit(X_train, y_train)
y_pred = lgbm_model2.predict(X_test)
# Calculate rmsle for a few example predictions 
y_true = y_test
n = len(features_train)
print('A couple of RMSLE scores computed over the train set')
print(f'Perfect prediction: {rmsle(y_true, y_true):.4f}')
print(f'Score: {rmsle(y_true, y_pred):.4f}')

print("Score: ",sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5)

c:\Users\47457\AppData\Local\Programs\Python\Python38\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


A couple of RMSLE scores computed over the train set
Perfect prediction: 0.0000
Score: 0.3392
Score:  0.3391502495397451
